In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import abc
import tensorflow as tf
import numpy as np
import time
import random
import time

import base64
import matplotlib.pyplot as plt
#import reverb
import tempfile

import logzero
from logzero import logger as logging

from tf_agents.environments import py_environment, batched_py_environment, tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.experimental.train import actor
from tf_agents.experimental.train import learner
from tf_agents.experimental.train import triggers
from tf_agents.experimental.train.utils import spec_utils
from tf_agents.experimental.train.utils import strategy_utils
from tf_agents.experimental.train.utils import train_utils
from tf_agents.metrics import py_metrics, tf_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy, py_tf_policy
from tf_agents.policies import random_py_policy, random_tf_policy
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.eval import metric_utils

from tf_agents.replay_buffers import tf_uniform_replay_buffer

from tf_agents.drivers import dynamic_step_driver, dynamic_episode_driver


from tf_agents.utils import common

from CM_py_env import CarMakerEnv
from server import Server

In [2]:
from tensorflow.python.client import device_lib
print (device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13152033329906518600
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11968989727887212644
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
#timestr = time.strftime("%Y%m%d-%H%M%S")
#root_dir = "%s/rl_data/run%s" % (os.getcwd(),  timestr)
#os.mkdir(root_dir)
root_dir = "/media/vmroot/Daten/RL_Metrics/run20201214-194710"

In [4]:
## Hyperparameter
num_iterations = 4000000 # @param {type:"integer"}

initial_collect_steps = 10000 # @param {type:"integer"}
train_steps_per_iteration = 1 # @param {type:"integer"}
replay_buffer_capacity = 100000 # @param {type:"integer"}
collect_actor_num_steps = 2
collect_actor_num_episodes = None # steps or episodes must be none

batch_size = 512 # @param {type:"integer"}

critic_learning_rate = 4e-4 # @param {typ"}ber"}
actor_learning_rate = 8e-4 # @param {type"}er"}
alpha_learning_rate = 4e-4 # @param {type"}er"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 15.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 100 # @param {type:"integer"}

num_eval_episodes = 3 # @param {type:"integer"}
eval_interval = 10000000000000000000 # @param {type:"integer"}

policy_save_interval = 10000 # @param {type:"integer"}

summary_interval=1000
summaries_flush_secs=10

eval_metrics_callback=None

train_checkpoint_interval=10000
policy_checkpoint_interval=10000
rb_checkpoint_interval=5000

In [5]:
logzero.logfile("%s/rotating-logfile.log" % root_dir, maxBytes=1e6, backupCount=3)

In [6]:
train_dir = os.path.join(root_dir, 'train')
eval_dir = os.path.join(root_dir, 'eval')

train_summary_writer = tf.compat.v2.summary.create_file_writer(
    train_dir, flush_millis=summaries_flush_secs * 1000)
train_summary_writer.set_as_default()

eval_summary_writer = tf.compat.v2.summary.create_file_writer(
    eval_dir, flush_millis=summaries_flush_secs * 1000)
eval_metrics = [
    tf_metrics.AverageReturnMetric(buffer_size=num_eval_episodes),
    tf_metrics.AverageEpisodeLengthMetric(buffer_size=num_eval_episodes)
]

In [7]:
# One Environment for collecting data during eval, and one for eval
#Collect langsamer

collect_env = CarMakerEnv(RTFac=1.7, mode='collect', gamma=gamma, server=0)
#collect_env2 = CarMakerEnv(RTFac=2, mode='collect', gamma=gamma, server=1)
#collect_env3 = CarMakerEnv(RTFac=1, mode='collect', gamma=gamma, server=2)
#collect_env4 = CarMakerEnv(RTFac=1, mode='collect', gamma=gamma, server=3)

#collect_env = batched_py_environment.BatchedPyEnvironment( [collect_env1, collect_env2] )
#eval_env = CarMakerEnv(RTFac=999999, mode='evaluate', gamma=gamma, server=1)

#eval_env = collect_env

tf_collect_env = tf_py_environment.TFPyEnvironment(collect_env)
tf_eval_env = tf_collect_env


In [8]:
# Check specs

print('Observation Spec:')
print(tf_collect_env.time_step_spec().observation)
print('Action Spec:')
print(tf_collect_env.action_spec())

Observation Spec:
TensorSpec(shape=(18,), dtype=tf.float32, name='observation')
Action Spec:
BoundedTensorSpec(shape=(2,), dtype=tf.float32, name='action', minimum=array([-1.  , -3.14], dtype=float32), maximum=array([1.  , 3.14], dtype=float32))


In [9]:
## Agents
# All variables and Agents need to be created under strategy.scope(), as you'll see below.

# Critic for estimate of action values
# Input is observation and an action
# Output is estimate of action value (to see how good it would be)

observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(tf_collect_env))


critic_net = critic_network.CriticNetwork(
      (observation_spec, action_spec),
      observation_fc_layer_params=None,
      action_fc_layer_params=None,
      joint_fc_layer_params=critic_joint_fc_layer_params,
      kernel_initializer='glorot_uniform',
      last_kernel_initializer='glorot_uniform')

In [10]:
# Actor network predicts parameters for a tanh-squashed MultivariateNormalDiag distribution.
# Will be sampled conditioned on the current state (observation), if generation of action is needed.


actor_net = actor_distribution_network.ActorDistributionNetwork(
    observation_spec,
    action_spec,
    fc_layer_params=actor_fc_layer_params,
    continuous_projection_net=(
        tanh_normal_projection_network.TanhNormalProjectionNetwork))

In [11]:
# Init agent

global_step = tf.compat.v1.train.get_or_create_global_step()

tf_agent = sac_agent.SacAgent(
    time_step_spec,
    action_spec,
    actor_network=actor_net,
    critic_network=critic_net,
    actor_optimizer=tf.compat.v1.train.AdamOptimizer(
        learning_rate=actor_learning_rate),
    critic_optimizer=tf.compat.v1.train.AdamOptimizer(
        learning_rate=critic_learning_rate),
    alpha_optimizer=tf.compat.v1.train.AdamOptimizer(
        learning_rate=alpha_learning_rate),
    target_update_tau=target_update_tau,
    target_update_period=target_update_period,
    td_errors_loss_fn=tf.math.squared_difference,
    gamma=gamma,
    reward_scale_factor=reward_scale_factor,
    train_step_counter=global_step)

tf_agent.initialize()


In [12]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    tf_agent.collect_data_spec,
    batch_size=1,
    max_length=replay_buffer_capacity)
# 6 instead of 2 for TD(5)

def _filter_invalid_transition(trajectories, unused_arg1):
      return ~trajectories.is_boundary()[0]

dataset = replay_buffer.as_dataset(
    sample_batch_size=batch_size, num_steps=2, num_parallel_calls=2).unbatch().filter(
            _filter_invalid_transition).batch(batch_size).prefetch(100)

replay_observer = [replay_buffer.add_batch]

iterator = iter(dataset)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [13]:
## Policies
# Agent has 2 policies

# agent.policy — The main policy that is used for evaluation and deployment.
# agent.collect_policy — A second policy that is used for data collection.

tf_eval_policy = greedy_policy.GreedyPolicy(tf_agent.policy)

tf_collect_policy = tf_agent.collect_policy

# Additional random policy for initial collector

random_policy = random_tf_policy.RandomTFPolicy(
  tf_collect_env.time_step_spec(), tf_collect_env.action_spec())
  

In [14]:
train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(
        buffer_size=num_eval_episodes, batch_size=tf_collect_env.batch_size),
    tf_metrics.AverageEpisodeLengthMetric(
        buffer_size=num_eval_episodes, batch_size=tf_collect_env.batch_size),
]

In [15]:
train_checkpointer = common.Checkpointer(
    ckpt_dir=train_dir,
    agent=tf_agent,
    global_step=global_step,
    metrics=metric_utils.MetricsGroup(train_metrics, 'train_metrics'))
policy_checkpointer = common.Checkpointer(
    ckpt_dir=os.path.join(train_dir, 'policy'),
    policy=tf_eval_policy,
    global_step=global_step)
rb_checkpointer = common.Checkpointer(
    ckpt_dir=os.path.join(train_dir, 'replay_buffer'),
    max_to_keep=1,
    replay_buffer=replay_buffer)

train_checkpointer.initialize_or_restore()
rb_checkpointer.initialize_or_restore()

In [16]:
# Actor with the random policy and collect experiences to seed the replay buffer with

initial_collect_actor = dynamic_step_driver.DynamicStepDriver(
  tf_eval_env,
  random_policy,
  num_steps=initial_collect_steps,
  observers=replay_observer+train_metrics)

In [17]:
# Instantiate an Actor with the collect policy to gather more experiences during training

# Steps per run nun 5 statt 1, weil smoother?


if collect_actor_num_episodes is not None and collect_actor_num_steps is not None:
  raise ValueError("Define num episodes OR num steps. One of them must be None.")
elif collect_actor_num_episodes is None:
  collect_actor = dynamic_step_driver.DynamicStepDriver(
    tf_collect_env,
    tf_collect_policy,
    num_steps=collect_actor_num_steps,
    observers=replay_observer+train_metrics)
elif collect_actor_num_steps is None:
  collect_actor = dynamic_episode_driver.DynamicEpisodeDriver(
    tf_collect_env,
    tf_collect_policy,
    num_episodes=collect_actor_num_episodes,
    observers=replay_observer+train_metrics)
  

In [18]:
initial_collect_actor.run = common.function(initial_collect_actor.run)
collect_actor.run = common.function(collect_actor.run)
tf_agent.train = common.function(tf_agent.train)

In [19]:
time_step = None
policy_state = tf_collect_policy.get_initial_state(tf_collect_env.batch_size)
timed_at_step = global_step.numpy()
time_acc = 0
global_episode_val = train_metrics[0].result().numpy()

In [20]:
if replay_buffer.num_frames() == 0:
    # Collect initial replay data.
    print(
        'Initializing replay buffer by collecting experience for %d steps '
        'with a random policy.' % initial_collect_steps)
    initial_collect_actor.run()

In [21]:
def train_step():
    experience, _ = next(iterator)
    return tf_agent.train(experience)

train_step = common.function(train_step)

In [22]:
global_step_val = global_step.numpy()

In [23]:
while global_step_val < num_iterations:
    start_time = time.time()
    time_step, policy_state = collect_actor.run(
        time_step=time_step,
        policy_state=policy_state,
    )
    Server(14100).server_nrt(str(global_episode_val))
    for _ in range(train_steps_per_iteration):
        train_loss = train_step()
    time_acc += time.time() - start_time

    global_step_val = global_step.numpy()
    global_episode_val = train_metrics[0].result().numpy()


    if global_step_val % log_interval == 0:
        logging.info('step = %d, episode = %d, loss = %f', global_step_val, global_episode_val, train_loss.loss)
        steps_per_sec = (global_step_val - timed_at_step) / time_acc
        logging.info('%.3f steps/sec', steps_per_sec)
        tf.compat.v2.summary.scalar(
            name='global_steps_per_sec', data=steps_per_sec, step=global_step)
        timed_at_step = global_step_val
        time_acc = 0

    for train_metric in train_metrics:
        train_metric.tf_summaries(
            train_step=global_step, step_metrics=train_metrics[:2])

    if global_step_val % eval_interval == 0:
        results = metric_utils.eager_compute(
            eval_metrics,
            tf_eval_env,
            tf_eval_policy,
            num_episodes=num_eval_episodes,
            train_step=global_step,
            summary_writer=eval_summary_writer,
            summary_prefix='Metrics',
        )
    if eval_metrics_callback is not None:
        eval_metrics_callback(results, global_step_val)
    metric_utils.log_metrics(eval_metrics)

    if global_step_val % train_checkpoint_interval == 0:
        train_checkpointer.save(global_step=global_step_val)

    if global_step_val % policy_checkpoint_interval == 0:
        policy_checkpointer.save(global_step=global_step_val)

    if global_step_val % rb_checkpoint_interval == 0:
        rb_checkpointer.save(global_step=global_step_val)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.


UnknownError:  AttributeError: 'Server' object has no attribute 'old_msg_roh'
Traceback (most recent call last):

  File "/home/vmroot/miniconda3/envs/tf_cuda/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 244, in __call__
    ret = func(*args)

  File "/home/vmroot/miniconda3/envs/tf_cuda/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 302, in wrapper
    return func(*args, **kwargs)

  File "/home/vmroot/miniconda3/envs/tf_cuda/lib/python3.8/site-packages/tf_agents/environments/tf_py_environment.py", line 318, in _isolated_step_py
    return self._execute(_step_py, *flattened_actions)

  File "/home/vmroot/miniconda3/envs/tf_cuda/lib/python3.8/site-packages/tf_agents/environments/tf_py_environment.py", line 214, in _execute
    return fn(*args, **kwargs)

  File "/home/vmroot/miniconda3/envs/tf_cuda/lib/python3.8/site-packages/tf_agents/environments/tf_py_environment.py", line 314, in _step_py
    self._time_step = self._env.step(pac

In [26]:
results = metric_utils.eager_compute(
    eval_metrics,
    tf_eval_env,
    tf_eval_policy,
    num_episodes=num_eval_episodes,
    train_step=global_step,
    summary_writer=eval_summary_writer,
    summary_prefix='Metrics',
)